<a href="https://colab.research.google.com/github/Alecia113/NLP-Ex/blob/main/check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import numpy as np 

from numpy import array
from numpy import argmax
from numpy import log

In [17]:
import re

# Pad sequences to the max length
def pad_sequences_pre(input_sequences, maxlen):
    output = []
    for inp in input_sequences:
        if len(inp)< maxlen:
            output.append([0]*(maxlen-len(inp)) + inp)
        else:
            output.append(inp[:maxlen])
    return output

# Prepare the data
def dataset_preparation(data):
    corpus = data.lower().split("\n")
    normalized_text=[]
    for string in corpus:
        tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
        normalized_text.append(tokens)
    tokenized_sentences=[sentence.strip().split(" ") for sentence in normalized_text]

    word_list_dict ={}
    for sent in tokenized_sentences:
        for word in sent:
            if word != "":
                word_list_dict[word] = 1
    word_list = list(word_list_dict.keys())
    word_to_index = {word:word_list.index(word) for word in word_list}

    total_words = len(word_list)+1

    # create input sequences using list of tokens
    input_sequences = []
    for line in tokenized_sentences:
        token_list = []
        for word in line:
            if word!="":
                token_list.append(word_to_index[word])
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)

    # pad sequences 
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences_pre(input_sequences, maxlen=max_sequence_len))

    # create predictors and label
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

    return predictors, np.array(label), max_sequence_len, total_words, word_list, word_to_index

data = '''The cat and her kittens
They put on their mittens
To eat a christmas pie
The poor little kittens
They lost their mittens
And then they began to cry.

O mother dear, we sadly fear
We cannot go to-day,
For we have lost our mittens
If it be so, ye shall not go
For ye are naughty kittens'''

predictors, label, max_sequence_len, total_words, word_list, word_to_index = dataset_preparation(data)


In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

# Define the model
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim_1, hidden_dim_2, total_words):
        super(LSTMTagger, self).__init__()
        self.hidden_dim_1 = hidden_dim_1
        self.hidden_dim_2 = hidden_dim_2
        self.word_embeddings = nn.Embedding(total_words, embedding_dim)
        self.lstm1 = nn.LSTM(embedding_dim, hidden_dim_1, batch_first=True)  
        self.lstm2 = nn.LSTM(hidden_dim_1, hidden_dim_2, batch_first=True)  
        self.hidden2tag = nn.Linear(hidden_dim_2, total_words)


    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out_1, _ = self.lstm1(embeds)
        lstm_out_2, _ = self.lstm2(lstm_out_1)
        tag_space = self.hidden2tag(lstm_out_2[:,-1,:])
        # The reason we are using log_softmax here is that we want to calculate -log(p) and find the minimum score                    
        tag_scores = F.log_softmax(tag_space, dim=1)      
        return tag_scores

# Parameter setting
EMBEDDING_DIM = 10
HIDDEN_DIM_1 = 150
HIDDEN_DIM_2 = 100
batch_size=predictors.shape[0]

model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM_1, HIDDEN_DIM_2, total_words).cuda()
loss_function = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


sentence =torch.from_numpy(predictors).cuda().to(torch.int64)
targets = torch.from_numpy(label).cuda().to(torch.int64)


# Training
for epoch in range(100):  

    model.train()
    model.zero_grad()       
    tag_scores = model(sentence)
    loss = loss_function(tag_scores, targets)
    loss.backward()
    optimizer.step()


    if epoch % 10 == 9:
        model.eval()
        _, predicted = torch.max(tag_scores, 1)
        prediction = predicted.view(-1).cpu().numpy()
        t = targets.view(-1).cpu().numpy()
        acc = accuracy_score(prediction,t)
        print('Epoch: %d, training loss: %.4f, training acc: %.2f%%'%(epoch+1,loss.item(),100*acc))



Epoch: 10, training loss: 3.6901, training acc: 10.42%
Epoch: 20, training loss: 3.4681, training acc: 10.42%
Epoch: 30, training loss: 3.0349, training acc: 14.58%
Epoch: 40, training loss: 2.6013, training acc: 29.17%
Epoch: 50, training loss: 2.2176, training acc: 47.92%
Epoch: 60, training loss: 1.8787, training acc: 72.92%
Epoch: 70, training loss: 1.6239, training acc: 77.08%
Epoch: 80, training loss: 1.3657, training acc: 89.58%
Epoch: 90, training loss: 1.1654, training acc: 89.58%
Epoch: 100, training loss: 0.9896, training acc: 93.75%


做后

In [19]:
# convert index to word
def ind_to_word(predicted_ind):
    for word, index in word_to_index.items():
        if index == predicted_ind:
            return word
    return ""    


# get the top k most predicted results
def get_topK(predicted, k): 
    
    # Get the index of the highest k index
    # Since the input is just one sentence, we can use [0] to extract the prediction result
    top_k = np.argsort(predicted[0])[-k:]

    # return a list of tuple
    # tuple[0]:word_id, tuple[1]:log(p)
    return [(id, predicted[0][id]) for id in top_k]


# To-Do: modify this function
# Generate text, currently only works with k=1 
# Hint: The easist way is modifying the code from line 40-47, but it is not compulsory

def generate_text(seed_text, next_words, max_sequence_len, k):

    seed_candidates = [(seed_text, .0)]
    for _ in range(next_words):
        successives = []
        for i in range(len(seed_candidates)):
            seed_text, score = seed_candidates[i]
            token_list = [word_to_index[word] for word in seed_text.split()]
            token_list = pad_sequences_pre([token_list], maxlen=max_sequence_len-1)

            seed_input = torch.from_numpy(np.array(token_list)).cuda().to(torch.int64)
            predicted = model(seed_input).cpu().detach().numpy()
            #加了个判断和循环
            if k == 1:
              id, s = get_topK(predicted, k)[0]
              output_word = ind_to_word(id)
              successives.append((seed_text + ' ' + output_word, score - s)) 
            else:
              id_n =[]
              s_n = []
              for j in range(k):            
                id_n.append(get_topK(predicted, k)[j][0])
                s_n.append(get_topK(predicted, k)[j][1])
                output_word = ind_to_word(id_n[j])   
                successives.append((seed_text + ' ' + output_word, score - s_n[j])) 

        ordered = sorted(successives, key=lambda tup: tup[1])
        seed_candidates = ordered[:k]

    return seed_candidates[0][0]   


print(generate_text("we naughty", 3, max_sequence_len, k=1))
print(generate_text("we naughty", 3, max_sequence_len, k=3))

# Please note that it can happen that k=1 and k=3 have the same output because this is only a small dataset.


we naughty go to day
we naughty go to day


lab 代码

In [20]:
# convert index to word
def ind_to_word(predicted_ind):
    for word, index in word_to_index.items():
        if index == predicted_ind:
            return word
    return ""    


# get the top k most predicted results
def get_topK(predicted, k=1):
    
    # Get the index of the highest k index
    # Since the input is just one sentence, we can use [0] to extract the prediction result
    top_k = np.argsort(predicted[0])[-k:]

    # return a list of tuple
    # tuple[0]:word_id, tuple[1]:log(p)
    return [(id, predicted[0][id]) for id in top_k]


# To-Do: modify this function
# Generate text, currently only works with k=1 
# Hint: The easist way is modifying the code from line 40-47, but it is not compulsory

def generate_text(seed_text, next_words, max_sequence_len, k=1):

    seed_candidates = [(seed_text, .0)]
    for _ in range(next_words):
        successives = []
        # if k = 1, len(seed_candidates) will always be 1
        for i in range(len(seed_candidates)):
            seed_text, score = seed_candidates[i]
            token_list = [word_to_index[word] for word in seed_text.split()]
            token_list = pad_sequences_pre([token_list], maxlen=max_sequence_len-1)

            seed_input = torch.from_numpy(np.array(token_list)).cuda().to(torch.int64)
            predicted = model(seed_input).cpu().detach().numpy()


            # Since it it only works with k = 1, we can simply use [0] to get the word id and log(p)
            # However, if k = 3, you can't simply use [0] to get the candidates
            id, s = get_topK(predicted, k)[0]
            # get the output word
            output_word = ind_to_word(id)
            # put the word into the sentence input
            # calcualte the accumulated score by -log(p)
            successives.append((seed_text + ' ' + output_word, score - s)) 

        # Get the lowest k accumulated scores (highest k accumulated probabilities)
        # Then, make them as the seed_candidate for the next word to predict
        ordered = sorted(successives, key=lambda tup: tup[1])
        seed_candidates = ordered[:k]

    return seed_candidates[0][0]


print(generate_text("we naughty", 3, max_sequence_len, k=1))
print(generate_text("we naughty", 3, max_sequence_len, k=3))

# Please note that it can happen that k=1 and k=3 have the same output because this is only a small dataset.


we naughty go to day
we naughty lost day to
